<a href="https://colab.research.google.com/github/ZwL6/MZT/blob/main/assignment-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u372-ga~us1-0ubuntu1~20.04).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
# Get cities file
!wget https://raw.githubusercontent.com/ZwL6/MZT/main/cities.json



--2023-07-02 13:03:45--  https://raw.githubusercontent.com/ZwL6/MZT/main/cities.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15829 (15K) [text/plain]
Saving to: ‘cities.json.1’

cities.json.1       100%[===================>]  15.46K  --.-KB/s    in 0s      

2023-07-02 13:03:45 (35.5 MB/s) - ‘cities.json.1’ saved [15829/15829]



In [ ]:
#making standerd route
import json
import random

# Set seed
random.seed(1)
with open('cities.json', 'r') as file:
    data = json.load(file)

cities = [d['city'] for d in data]

merchandise = ['apple', 'tomatoes', 'butter', 'water', 'milk', 'pens', 'honey']

routes = []

for i in range(1000):
    route = {}
    route['id'] = i

    # Generate a list of non-repeating random cities
    city_sample = random.sample(cities, 3)

    # Randomly generate the number of merchandise
    num_merchandise = random.randint(1, 10)

    # Randomly select merchandise types
    merchandise_sample = random.sample(merchandise, random.randint(3, 5))

    # Build the route dictionary format
    route['route'] = []
    for j in range(2):
        route_info = {}
        route_info['from'] = city_sample[j]
        route_info['to'] = city_sample[j+1]

        # Generate random merchandise quantities for each route
        merchandise_dict = {}
        for m in merchandise_sample:
            merchandise_dict[m] = random.randint(1, num_merchandise)

        route_info['merchandise'] = merchandise_dict

        route['route'].append(route_info)

    routes.append(route)

# Write the route list to a JSON file without corrupt records
valid_routes = [r for r in routes if 'route' in r and len(r['route']) == 2]
with open('routes.json', 'w') as file:
    json.dump(routes, file, indent=4)


In [ ]:
# Download routes
# from google.colab import files
# files.download('routes.json')


In [ ]:
# making actual route

import json
import random
import copy

# Set seed
random.seed(2)

# Read the JSON file
with open('cities.json', 'r') as file:
    data = json.load(file)

# Extract city names
cities = [item['city'] for item in data]

# Define merchandise types
merchandise_types = ['apple', 'tomatoes', 'butter', 'water', 'milk', 'pens', 'honey']

with open('routes.json', 'r') as file:
    routes = json.load(file)

act_routes = []

for route in routes:
    if len(route['route']) < 2:
        continue  # Skip routes with less than two cities

    for _ in range(10):
        act_route = copy.deepcopy(route)  # Use deepcopy to avoid modifying the original route

        # Randomly select a city to modify
        modified_city_index = random.randint(0, len(act_route['route']) - 1)
        modified_city = act_route['route'][modified_city_index]

        # Randomly decide whether to modify city
        if random.choice([True, False]):
            # Modify the selected city
            new_city = random.choice(cities)
            modified_city['from'] = new_city

            # If the modified city is the second city, modify it throughout the route
            if modified_city_index == 1:
                # Replace the modified city with a new city
                act_route['route'][modified_city_index]['from'] = new_city
                # Connect the cities before and after the replaced city
                act_route['route'][modified_city_index - 1]['to'] = new_city
                if modified_city_index + 1 < len(act_route['route']):
                    act_route['route'][modified_city_index]['to'] = act_route['route'][modified_city_index + 1]['from']

        # Randomly modify merchandise
        for city_route in act_route['route']:
          # Randomly decide whether to modify merchandise
          if random.choice([True, False]):
            # Select a random merchandise type
            modified_merchandise = random.choice(list(city_route['merchandise'].keys()))
            # Modify the quantity or replace the merchandise type
            if random.choice([True, False]):
              # Modify the quantity
              city_route['merchandise'][modified_merchandise] = random.randint(1, 5)
            else:
              # Replace the merchandise type
              new_merchandise = random.choice(merchandise_types)
              if modified_merchandise in city_route['merchandise']:
                city_route['merchandise'][new_merchandise] = city_route['merchandise'][modified_merchandise]
                del city_route['merchandise'][modified_merchandise]


        # Randomly modify merchandise
        #for city_route in act_route['route']:
            # Select random merchandise types
            #selected_merchandise = random.sample(merchandise_types, random.randint(1, len(merchandise_types)))
            # Assign random quantities to selected merchandise
            #city_route['merchandise'] = {item: random.randint(1, 5) for item in selected_merchandise}

        act_routes.append(act_route)

# Write the act routes list to a new JSON file
with open('act_routes.json', 'w') as file:
    json.dump(act_routes, file, indent=4)

In [ ]:
# Download the actual routes
# from google.colab import files
# files.download('act_routes.json')

In [ ]:
!pip install python-Levenshtein
import json
from collections import defaultdict
from Levenshtein import distance

# Function for calculating route edit distance similarity
def calculate_similarity(route1, route2):
    lev_distance = distance(route1, route2)
    max_length = max(len(route1), len(route2))
    similarity = 1 - lev_distance / max_length
    return similarity

# Function for calculating marchandise edit distance similarity
def calculate_merchandise_similarity(merch1, merch2):
    lev_distance = distance(merch1, merch2)
    max_length = max(len(merch1), len(merch2))
    similarity = 1 - lev_distance / max_length
    return similarity

# open standard route file
with open('routes.json', 'r') as file:
    routes = json.load(file)

# open actual route file
with open('act_routes.json', 'r') as file:
    act_routes = json.load(file)

# Create a dictionary to store the most similar route for each actual route
most_similar_routes = defaultdict(list)

# Correct prediction counter
correct_predictions = 0

# Iterate through all actual routes
for act_route in act_routes:
    act_route_id = act_route['id']
    act_route_str = ' '.join([f"{route['from']} {route['to']}" for route in act_route['route']])

    similarities = []

    # route similarity calculating
    for standard_route in routes:
        standard_route_id = standard_route['id']
        standard_route_str = ' '.join([f"{route['from']} {route['to']}" for route in standard_route['route']])

        route_similarity = calculate_similarity(standard_route_str, act_route_str)

        # merchandise similarity calculating
        merchandise_similarity = 0
        for i, route in enumerate(act_route['route']):
            act_merchandise = route['merchandise']
            standard_merchandise = standard_route['route'][0]['merchandise']
            merchandise_similarity += calculate_merchandise_similarity(str(standard_merchandise), str(act_merchandise))
        merchandise_similarity /= len(standard_route['route'])

        #total similarity calculating
        total_similarity = (route_similarity + merchandise_similarity) / 2
        similarities.append((standard_route_id, total_similarity))

    # sort the max similarity
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Find the most similar route
    max_similarity = similarities[0]

    # Store the most similar route for this actual route
    most_similar_routes[act_route_id].append(max_similarity)

    # If the actual route id is the same as the predicted route id, increment the correct prediction counter
    if act_route_id == max_similarity[0]:
        correct_predictions += 1

# Print the most similar routes for all actual routes
# for act_route_id, max_similarities in most_similar_routes.items():
#    for i, max_similarity in enumerate(max_similarities):
#        print(f"Max similarity for actual route {act_route_id}-{i+1}: Standard Route {max_similarity[0]} with similarity {max_similarity[1]}")

# Calculate and print the prediction accuracy
accuracy = correct_predictions / len(act_routes)
print(f"Prediction accuracy: {accuracy * 100}%")



Prediction accuracy: 99.2%


In [ ]:
# Convert 'standard routes' file to Spark DataFrame
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

# Load data from JSON file
with open('routes.json', 'r') as file:
    data = json.load(file)

# Initialize an empty DataFrame to store the processed data
df = pd.DataFrame()

# Iterate over each element
for element in data:
  id = element['id']

# Iterate over routes
  for route in element['route']:
    from_city = route['from']
    to_city = route['to']

    # Iterate over merchandise
    for merchandise, quantity in route['merchandise'].items():
        # Append data to DataFrame
        df = df.append({
            'id': id,
            'from': from_city,
            'to': to_city,
            'merchandise': merchandise,
            'quantity': quantity
        }, ignore_index=True)
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Convert DataFrame to Spark DataFrame
st_routes_df = spark.createDataFrame(df)

In [ ]:
from pyspark.sql.functions import expr

# Create a new column that is the merchandise column value repeated quantity times
str_df = st_routes_df.withColumn('merchandise_quantity', expr("repeat(merchandise, quantity)"))
str_df = str_df.drop('merchandise','quantity')

str_df.show(5)

+---+-----------+-----------+--------------------+
| id|       from|         to|merchandise_quantity|
+---+-----------+-----------+--------------------+
|  0|    Helmond|Schoonhoven|waterwaterwaterwater|
|  0|    Helmond|Schoonhoven|          honeyhoney|
|  0|    Helmond|Schoonhoven|                pens|
|  0|Schoonhoven|    Zaandam|waterwaterwaterwater|
|  0|Schoonhoven|    Zaandam|               honey|
+---+-----------+-----------+--------------------+
only showing top 5 rows



In [ ]:
#STANDERD ROUTE
from pyspark.sql.functions import concat_ws, collect_list

# Group the dataframe by 'id', 'from', and 'to' and concatenate the 'merchandise_quantity' strings
st_df = str_df.groupBy('id', 'from', 'to').agg(concat_ws('', collect_list('merchandise_quantity')).alias('combined_merchandise'))

st_df.show(5)

+---+------------+------------+--------------------+
| id|        from|          to|combined_merchandise|
+---+------------+------------+--------------------+
|  0|     Helmond| Schoonhoven|waterwaterwaterwa...|
|  0| Schoonhoven|     Zaandam|waterwaterwaterwa...|
|  1|Klazienaveen|   The Hague|penstomatoestomat...|
|  1|      Nuenen|Klazienaveen|penspenspenspensp...|
|  2|    Enschede|   Harlingen|milkmilkmilkmilkt...|
+---+------------+------------+--------------------+
only showing top 5 rows



In [ ]:
import pandas as pd
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id

# Load data from JSON file
with open('act_routes.json', 'r') as file:
    data = json.load(file)

# Initialize an empty DataFrame to store the processed data
df = pd.DataFrame()

# Dictionary to store the counter for each unique ID
id_counter = {}

# Iterate over each element
for element in data:
    id = element['id']

    # Increment the counter for each unique ID
    if id in id_counter:
        id_counter[id] += 1
    else:
        id_counter[id] = 1

    # Counter for each ID
    counter = id_counter[id]

    # Iterate over routes
    for route in element['route']:
        from_city = route['from']
        to_city = route['to']

        # Iterate over merchandise
        for merchandise, quantity in route['merchandise'].items():
            # Append data to DataFrame with modified ID
            df = df.append({
                'ac_id': f"{id}-{counter}",  # Assign modified ID with counter
                'original_id': id,  # Store original ID
                'from': from_city,
                'to': to_city,
                'merchandise': merchandise,
                'quantity': quantity
            }, ignore_index=True)

# Convert DataFrame to Spark DataFrame
spark = SparkSession.builder.getOrCreate()
ac_routes_df = spark.createDataFrame(df)

# Assign sequential counts to duplicate IDs
ac_routes_df = ac_routes_df.withColumn('new_id', monotonically_increasing_id())


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Calculate the sum of quantity for each ac_id and original_id
acpath_df = ac_routes_df.groupBy('ac_id', 'original_id').agg(F.sum('quantity').alias('ac_quantity'))

# Sort the DataFrame by ac_id
acpath_df = acpath_df.orderBy('ac_id')

# Show the resulting DataFrame
acpath_df.show()

+-----+-----------+-----------+
|ac_id|original_id|ac_quantity|
+-----+-----------+-----------+
|  0-1|          0|         16|
| 0-10|          0|         16|
|  0-2|          0|         14|
|  0-3|          0|         12|
|  0-4|          0|         16|
|  0-5|          0|         12|
|  0-6|          0|         16|
|  0-7|          0|         12|
|  0-8|          0|         16|
|  0-9|          0|         19|
|  1-1|          1|         26|
| 1-10|          1|         21|
|  1-2|          1|         26|
|  1-3|          1|         22|
|  1-4|          1|         26|
|  1-5|          1|         27|
|  1-6|          1|         27|
|  1-7|          1|         28|
|  1-8|          1|         28|
|  1-9|          1|         28|
+-----+-----------+-----------+
only showing top 20 rows



In [ ]:
# combine merchandies and quantity
acr_df = ac_routes_df.withColumn('merchandise_quantity', expr("repeat(merchandise, quantity)"))
acr_df = acr_df.drop('merchandise', 'quantity', 'new_id')

acr_df.show(5)


+-----+-----------+-----------+-----------+--------------------+
|ac_id|original_id|       from|         to|merchandise_quantity|
+-----+-----------+-----------+-----------+--------------------+
|  0-1|          0|Barendrecht|Schoonhoven|waterwaterwaterwater|
|  0-1|          0|Barendrecht|Schoonhoven|          honeyhoney|
|  0-1|          0|Barendrecht|Schoonhoven|                pens|
|  0-1|          0|Schoonhoven|    Zaandam|waterwaterwaterwater|
|  0-1|          0|Schoonhoven|    Zaandam|               honey|
+-----+-----------+-----------+-----------+--------------------+
only showing top 5 rows



In [ ]:
#ACTUAL ROUTE
from pyspark.sql.functions import first

# Group the dataframe by 'id', 'original_id', 'from', and 'to' and concatenate the 'merchandise_quantity' strings
ac_df = acr_df.groupby('ac_id', 'original_id', 'from', 'to').agg(concat_ws(' ', collect_list('merchandise_quantity')).alias('combined_merchandise'))

ac_df.show(5)

+-----+-----------+-----------+-----------+--------------------+
|ac_id|original_id|       from|         to|combined_merchandise|
+-----+-----------+-----------+-----------+--------------------+
|  0-1|          0|Schoonhoven|    Zaandam|waterwaterwaterwa...|
|  0-2|          0|     Bergen|Schoonhoven|waterwaterwaterwa...|
|  0-4|          0|    Helmond|Schoonhoven|waterwaterwaterwa...|
|  0-5|          0|Schoonhoven|    Zaandam|waterwaterwaterwa...|
|  0-5|          0|  Staphorst|Schoonhoven|waterwaterwaterwa...|
+-----+-----------+-----------+-----------+--------------------+
only showing top 5 rows



In [ ]:
!pip install python-Levenshtein
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType
from pyspark.sql.functions import pandas_udf, PandasUDFType, col
import pandas as pd
import Levenshtein

# Define a pandas UDF to calculate similarity
@pandas_udf(returnType=FloatType(), functionType=PandasUDFType.SCALAR)
def calculate_similarity(str1: pd.Series, str2: pd.Series) -> pd.Series:
    similarity = str1.combine(str2, lambda s1, s2: 1 - Levenshtein.distance(s1, s2) / max(len(s1), len(s2)))
    return similarity

# Rename the columns 'from' and 'to' in ac_df and st_df
ac_df = ac_df.withColumnRenamed('from', 'ac_from').withColumnRenamed('to', 'ac_to')
st_df = st_df.withColumnRenamed('from', 'st_from').withColumnRenamed('to', 'st_to')
a_df = ac_df.select(col('ac_id'), col('original_id'), col('ac_from'), col('ac_to'), col('combined_merchandise'))
s_df = st_df.select(col('id'), col('st_from'), col('st_to'), col('combined_merchandise'))

# Combine the actual routes with the same act_id
a_df = a_df.groupBy('ac_id').agg(F.first('original_id').alias('original_id'), F.concat_ws(' ', F.collect_list('ac_from'), F.collect_list('ac_to'), F.collect_list('combined_merchandise')).alias('combined_route'))

# Combine the standard routes with the same id
s_df = s_df.groupBy('id').agg(F.concat_ws(' ', F.collect_list('st_from'), F.collect_list('st_to'), F.collect_list('combined_merchandise')).alias('st_combined_route'))

# Calculate similarity using pandas UDF
ac_st_df = a_df.crossJoin(s_df).withColumn('similarity', calculate_similarity(col('combined_route'), col('st_combined_route')))

# Find the most similar route for each actual route
window = Window.partitionBy(ac_st_df['ac_id']).orderBy(ac_st_df['similarity'].desc())
most_similar_routes = ac_st_df.withColumn('rank', F.row_number().over(window)).filter(col('rank') == 1).select('original_id', 'id', 'similarity')

# Count the correct predictions
correct_predictions = most_similar_routes.filter(col('original_id') == col('id')).count()

# Calculate and print the prediction accuracy
accuracy = correct_predictions / ac_df.count()
print(f"Prediction accuracy: {accuracy * 100}%")


Prediction accuracy: 41.715%


In [ ]:
most_similar_routes = ac_st_df.withColumn('rank', F.row_number().over(window)).filter(col('rank') == 1).select('ac_id', 'similarity')
most_similar_routes.show(11)

+-----+----------+
|ac_id|similarity|
+-----+----------+
|  0-8|0.56666666|
|  0-9|0.85925925|
|  1-1| 0.8367347|
|  1-2| 0.9132653|
|  1-3| 0.5102041|
|  1-7| 0.5648148|
|  1-8|0.97029704|
|10-10| 0.6280992|
| 10-6|0.57024795|
| 10-7| 0.8429752|
| 10-8| 0.5068493|
+-----+----------+
only showing top 11 rows



In [ ]:
most_similar_routes.count()

10000

In [ ]:
# Join the acpath DataFrame with most_similar_routes based on ac_id
acpath_with_similarity = acpath_df.join(most_similar_routes, on='ac_id', how='left')

# Show the resulting DataFrame
acpath_with_similarity.show()


+------+-----------+-----------+----------+
| ac_id|original_id|ac_quantity|similarity|
+------+-----------+-----------+----------+
|117-10|        117|         44| 0.5745192|
| 118-8|        118|         28| 0.6089386|
| 200-5|        200|         27| 0.5804598|
| 209-6|        209|          9|       0.5|
| 220-1|        220|         10| 0.8043478|
| 225-6|        225|         16|0.70542634|
| 264-5|        264|          6|       0.6|
| 278-2|        278|         10| 0.7234042|
| 290-8|        290|         16| 0.9054054|
|  32-6|         32|         24|0.61403507|
|  62-1|         62|         30| 0.7821782|
|  66-5|         66|          6| 0.5135135|
| 101-8|        101|         27| 0.9268293|
|  11-3|         11|         23| 0.5620915|
| 126-2|        126|         13| 0.8956522|
|  18-1|         18|         18| 0.7222222|
| 214-7|        214|         26|0.86413044|
| 222-7|        222|         17|0.95620435|
| 225-3|        225|         13| 0.6764706|
|  39-2|         39|         32|

In [ ]:
# Define constants
BONUS_AMOUNT = 1000
ROUTE_AMOUNT = 250
MERCHANDISE_RATE = 2
BANK_ACCOUNT_LIMIT = 500000
FINE_PERCENTAGE = 0.05

from pyspark.sql.functions import lit, sum as Fsum, row_number
from pyspark.sql.window import Window

# Calculate payment for each driver in actual routes
def calculate_payment(act_routes):
    # Calculate the payment using Spark DataFrame functions
    act_routes = act_routes.withColumn('payment', (act_routes['similarity'] * lit(BONUS_AMOUNT)) + lit(ROUTE_AMOUNT * 2) + (act_routes['ac_quantity'] * lit(MERCHANDISE_RATE)))
    total_payment = act_routes.select(Fsum('payment')).first()[0]

    return total_payment, act_routes

# Optimize payment scheme
def optimize_payment(act_routes):
    # Create a window specification
    window = Window.orderBy(act_routes['payment'].desc())

    # Add a cumulative sum column
    act_routes = act_routes.withColumn('cumulative_payment', Fsum('payment').over(window))

    # Calculate the number of routes that can be paid
    max_payment = act_routes.filter(act_routes['cumulative_payment'] <= BANK_ACCOUNT_LIMIT).select(Fsum('payment')).first()[0]

    # Filter the routes that can be paid
    paid_routes = act_routes.filter(act_routes['payment'] <= max_payment)

    # Calculate the deposit back amount
    deposit_back = BANK_ACCOUNT_LIMIT - max_payment

    # Calculate the fine amount
    fine = deposit_back * FINE_PERCENTAGE

    return deposit_back, fine, paid_routes

# Calculate total payment amount and update the DataFrame with 'payment' column
total_payment, acpath_with_similarity = calculate_payment(acpath_with_similarity)

# Optimize payment scheme and get deposit back amounts, fine amount, and payment history
deposit_back_amount, fine_amount, paid_routes = optimize_payment(acpath_with_similarity)

# Print the results
print(f"Total payment amount: ${total_payment:.2f}")
print(f"Total deposit back amount: ${deposit_back_amount:.2f}")
print(f"Total fine amount: ${fine_amount:.2f}")

Total payment amount: $13051630.07
Total deposit back amount: $120.42
Total fine amount: $6.02
